# Preparing the launch file

In [1]:
import numpy as np
import pandas as pd


In [108]:
#pd.read_csv("./source-db/mp-160/POSCAR-unitcell")

pos_path = './source-db/mp-160/POSCAR-unitcell'
sample_path = './launch_thermal_si.py'
pos_file = open(pos_path,"r")

scaled_coordinates=open('./source-db/mp-160/scaled_coordinates.dat', "w")
linenumber=0
for line in pos_file.readlines():
    linenumber=linenumber+1
    if 3<=linenumber<=5:
        print("[",line,"],")


    if linenumber>=9:
        scaled_coordinates.write("("+line+"),\n")
    

scaled_coordinates.close()

[      4.8786175399999996    0.0000000000000000    0.0000000000000000
 ],
[     -2.4393087699999998    4.2250067299999996    0.0000000000000000
 ],
[      0.0000000000000000    0.0000000000000000   12.5104310900000009
 ],


# Parallel phono3py calculations using openMP

Togo-sensei found my phono3py in boston-lab did not use the parallel thread. I tried restalled phonopy and phono3py for several times, but didn' figure it out.

Today, after checking my .bashrc file line by line, I found the problem is caused my setting for the number of thread.

export OMP_NUM_THREADS = 1

After removing this line, and source .bashrc. The threads can be parallel.

## An example to get lattice thermal conductivity

be sure you have setup a virtual environment for aiida, and have a work job with its pk number (e.g. 186270 in my example)

In [ ]:
open verdi shell

In [ ]:
n3 = load_node(186270)

In [ ]:
n3.out.kappa

In [ ]:
<ArrayData: uuid: af240b4c-29ad-4c80-8691-a52a65d1587d (pk: 187185)>

In [ ]:
n3.out.kappa.get_arraynames()

In [ ]:
Out[4]:
[u'group_velocity',
 u'temperature',
 u'mesh',
 u'heat_capacity',
 u'frequency',
 u'gv_by_gv',
 u'gamma',
 u'weight',
 u'kappa',
 u'kappa_unit_conversion',
 u'qpoint',
 
 u'mode_kappa']

In [ ]:
f = n3.out.kappa

In [ ]:
f.get_array('kappa')[30] #this will print the ltc tensors

# output to json file example

In [3]:
import json
data = {
    'name': 'TiO2',
    'ID': 'mp-100',
    'ltc': [100,200]
}
json_str = json.dumps(data)

In [4]:
json_str

'{"name": "TiO2", "ID": "mp-100", "ltc": [100, 200]}'

In [ ]:
70325

# A practical example

In [2]:
import json
from pymatgen.matproj.rest import MPRester
import os
data = {}
#structure_id is the ID of the structure in Materials Project


raster = MPRester(os.environ['PMG_MAPI_KEY'])
def basic_info_from_mp(structure_id):
    material_name = raster.query(structure_id,['pretty_formula'])
#    mg_structure = raster.get_structure_by_material_id(structure_id)
#    material_name = mg_structure.formula.replace('1', '').replace(' ', '')
    return(structure_id, material_name[0]['pretty_formula'])

data['compound'] = []
data['compound'].append({
    'structure_id': 'mp-149',
    'material_name': basic_info_from_mp('mp-149')[1]
})

with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [3]:
%cat data.json

{"compound": [{"structure_id": "mp-149", "material_name": "Si"}]}

In [4]:
import json
from pymatgen.matproj.rest import MPRester
import os
data = {}
#structure_id is the ID of the structure in Materials Project


raster = MPRester(os.environ['PMG_MAPI_KEY'])
def basic_info_from_mp(structure_id):
    material_name = raster.query(structure_id,['pretty_formula'])
    return(structure_id, material_name[0]['pretty_formula'])

structure_id_list = ['mp-149','mp-1000']
data['compound'] = []
for i in structure_id_list:
    data['compound'].append({
        'structure_id': i,
        'material_name': basic_info_from_mp(i)[1]
    })

# data['compound'] = []
# data['compound'].append({
#     'structure_id': 'mp-149',
#     'material_name': name_in_mp('mp-149')
# })

with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [5]:
%cat data.json

{"compound": [{"structure_id": "mp-149", "material_name": "Si"}, {"structure_id": "mp-1000", "material_name": "BaTe"}]}

In [ ]:
mp-149, silicon, pk=197320
mp-463,
mp-286,
mp-830,
mp-22862,pk=91699
mp-1000, pk=85993
mp-422  pk=187245
mp-252 pk=187311 BeTe #folder in bridges
mp-971, pk=85957
mp-682, pk=86249
mp-1087, pk=87250  Attention: the coordinates order are changed althought it still suits for the elements
mp-2472, pk=87854
mp-2605, pk=87992
mp-7140, pk=88463
mp-981, pk=88586
mp-1394, pk=88747
mp-1138, pk=89733
mp-23193, pk=90390
mp-1315  pk=190715 MgS
mp-1265 pk=190837  MgO